# 评价xx模型预测结果的hydrological feature

## 导入依赖

In [1]:
import json
import os
import sys
import pickle

import numpy as np
import pandas as pd
from dotenv import load_dotenv

load_dotenv()
sys.path.append(os.getenv("PROJ_PATH")) # type: ignore

from dmg.core.post.plot_baseflowindex_scatter import \
    plot_baseflow_scatter  # noqa
from project.better_estimate import load_config  # noqa

## 导入配置

In [2]:
config = load_config(r"conf/config_dhbv_lstm.yaml")
out_path = config["out_path"]

## 读取数据

### 读取模型预测结果

In [3]:
predict_flow = np.load(
    os.path.join(out_path, "streamflow.npy"), allow_pickle=True
)

### 读取模型输入的降雨量

In [14]:
from dmg.core.data.loaders import HydroLoader  # noqa

loader = HydroLoader(config, test_split=True, overwrite=False) # type: ignore
loader.load_dataset()
eval_dataset = loader.eval_dataset

# forcing_data = eval_dataset['x_phy'].detach().cpu().numpy()[365:5110+365, :, :]
forcing_data = eval_dataset['x_phy'].detach().cpu().numpy()[:, :, :]
print("Forcing data shape:", forcing_data[0:100,0,:])
sum_prcp = forcing_data[:, :, 0].sum(axis=0)  # 累计降雨量

Forcing data shape: [[  0.          11.56         2.054303  ]
 [  0.          12.92         2.045786  ]
 [  5.96        14.22         1.740576  ]
 [ 31.3         11.46         1.5446584 ]
 [  2.39         9.405        1.6123645 ]
 [  0.           9.495        1.2110019 ]
 [  7.06         8.525        1.1104783 ]
 [  0.           7.645        1.5073547 ]
 [  0.           9.595        1.6177741 ]
 [  0.           4.875        1.6237459 ]
 [ 15.2          5.38         1.6349796 ]
 [ 23.36         9.795        1.4814948 ]
 [  0.           7.09         1.5865564 ]
 [  0.           4.59         1.0986187 ]
 [  0.           1.855        1.150855  ]
 [  0.           2.675        1.3247793 ]
 [  0.           5.42         1.4407805 ]
 [  6.01         8.275        1.6205635 ]
 [  9.          12.405        1.2903825 ]
 [  0.           9.89         1.4560266 ]
 [  0.           5.085        1.418569  ]
 [  0.           5.01         1.1887254 ]
 [  0.           3.28         1.1515857 ]
 [  0.        

## 计算预测结果的水文特征

### 计算预测结果平均值

In [5]:
predict_qmean = np.mean(predict_flow, axis=0)

### 计算runoff ratio

In [6]:
print("Sum precipitation sample:", sum_prcp.shape)
runoff_ratio_lstm = predict_qmean / sum_prcp

Sum precipitation sample: (531,)


### 计算Slope FDC

In [10]:
q33_obs = np.quantile(predict_flow, 0.67, axis=0) # 结果 shape: (N,)
q66_obs = np.quantile(predict_flow, 0.34, axis=0) # 结果 shape: (N,)

slope_fdc_sim = ((np.log(q33_obs) - np.log(q66_obs)) / (0.66 - 0.33))[:,0]
print("Slope FDC sample:", slope_fdc_sim.shape)

Slope FDC sample: (531,)
